# Generate Choice file for all participants

In [24]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
# Useful to quickly extract csv files
import glob
import os

In [25]:
# Extract Rate data

In [26]:
# Load data all participants
rateNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/*_RateData_PFT.csv')
choiceNameFrames = glob.glob('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/*_ChoiceData_PFT.csv')

rateNameFrames.sort()
choiceNameFrames.sort()

In [27]:
rateNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S123456_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S1_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S2_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S3_RateData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S4_RateData_PFT.csv']

In [28]:
choiceNameFrames

['/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S123456_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S1_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S2_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S3_ChoiceData_PFT.csv',
 '/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/S4_ChoiceData_PFT.csv']

## Extremes of rating scale in this task : 1 is min, 50 is max

In [30]:
partic_num=len(rateNameFrames) #number of participants

choiceFrameAll = pd.DataFrame( columns = ['trial','leftId','rightId','Frame','endTime','choice','choiceTime','LRat1','RRat1','LRat2','RRat2','LVar1','RVar1','LVar2','RVar2','Part'])

for j in range (partic_num):
    rateFrame = pd.read_csv(rateNameFrames[j])
    choiceFrames = pd.read_csv(choiceNameFrames[j])
    
    if 'ratL' in choiceFrames.columns:
        if 'ratR' in choiceFrames.columns:
            choiceFrames = choiceFrames.drop(['ratL', 'ratR'], axis=1)

    
    idNames = list(rateFrame.picId.unique())
    IdRatings1 = [None]*len(idNames)
    IdRatings2 = [None]*len(idNames)
    IdRatingsVar1 = [None]*len(idNames)
    IdRatingsVar2 = [None]*len(idNames)

    
    #extract average rating for like, dislike
    for i in range(len(idNames)):
        IdRatings1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.mean()
        IdRatings2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.mean()
        
        IdRatingsVar1[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 1)].rating.std()
        IdRatingsVar2[i] = rateFrame.loc[(rateFrame['picId'] == idNames[i]) & (rateFrame['frame'] == 2)].rating.std()

    rateValues = pd.DataFrame( {'nameId': idNames,'rat1': IdRatings1,'rat2': IdRatings2,'var1': IdRatingsVar1,'var2': IdRatingsVar2,})
    # separate ratings and rating-variability for each frame
    ChoiceLRat1 = [None]*len(choiceFrames)
    ChoiceRRat1 = [None]*len(choiceFrames)
    ChoiceLRat2 = [None]*len(choiceFrames)
    ChoiceRRat2 = [None]*len(choiceFrames)

    ChoiceLVar1 = [None]*len(choiceFrames)
    ChoiceRVar1 = [None]*len(choiceFrames)
    ChoiceLVar2 = [None]*len(choiceFrames)
    ChoiceRVar2 = [None]*len(choiceFrames)
    
    for i in range(len(choiceFrames)):

        choiceL = choiceFrames.iloc[i].leftId
        choiceR = choiceFrames.iloc[i].rightId
        
        ChoiceLRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat1.values[0]
        ChoiceRRat1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat1.values[0]
        ChoiceLRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].rat2.values[0]
        ChoiceRRat2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].rat2.values[0]
        
        ChoiceLVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var1.values[0]
        ChoiceRVar1[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var1.values[0]
        ChoiceLVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceL)].var2.values[0]
        ChoiceRVar2[i] = rateValues.loc[(rateValues['nameId'] == choiceR)].var2.values[0]

    choiceFrames['LRat1'] = ChoiceLRat1
    choiceFrames['RRat1'] = ChoiceRRat1
    choiceFrames['LRat2'] = ChoiceLRat2
    choiceFrames['RRat2'] = ChoiceRRat2 
    
    choiceFrames['LVar1'] = ChoiceLVar1
    choiceFrames['RVar1'] = ChoiceRVar1
    choiceFrames['LVar2'] = ChoiceLVar2
    choiceFrames['RVar2'] = ChoiceRVar2
     
    choiceFrames['Part'] = [j]*len(choiceFrames)    

    choiceFrameAll = pd.concat([choiceFrameAll,choiceFrames],ignore_index=True)
    

IndexError: index 0 is out of bounds for axis 0 with size 0

# Save file

In [7]:
choiceFrameAll.to_csv('/Users/pradyumna/Documents/GiTs/PFT_fMRI_2019/Pilots/Data/All/PFT_Choice_Pilot_All.csv',index = False)

In [8]:
choiceFrameAll

,trial,leftId,rightId,Frame,endTime,choice,choiceTime,LRat1,RRat1,LRat2,RRat2,LVar1,RVar1,LVar2,RVar2,Part
0,0.0,19.0,8.0,2.0,7.507573,1.0,1.580314,17.0,41.0,36.0,7.0,19.798990,11.313708,4.242641,1.414214,0
1,1.0,10.0,16.0,1.0,7.516446,NaN,NaN,31.0,46.0,6.0,30.5,0.000000,1.414214,7.071068,27.577164,0
2,2.0,11.0,5.0,1.0,7.502563,NaN,NaN,49.0,8.5,33.0,29.5,0.000000,3.535534,0.000000,27.577164,0
3,3.0,20.0,24.0,2.0,7.500934,1.0,0.113674,19.0,36.0,19.5,49.0,11.313708,4.242641,19.091883,0.000000,0
4,4.0,21.0,25.0,1.0,7.501782,1.0,0.026090,49.0,32.0,32.0,20.0,0.000000,1.414214,1.414214,18.384776,0
5,5.0,13.0,14.0,2.0,7.502454,1.0,0.529314,33.0,19.5,30.0,19.5,0.000000,19.091883,26.870058,19.091883,0
6,6.0,6.0,2.0,2.0,7.502645,-1.0,0.129977,41.0,40.0,28.5,17.0,11.313708,12.727922,28.991378,22.627417,0
7,7.0,4.0,19.0,1.0,7.500512,1.0,0.630875,27.0,17.0,23.5,36.0,22.627417,19.798990,13.435029,4.242641,0
8,8.0,6.0,2.0,1.0,7.501672,1.0,0.282406,41.0,40.0,28.5,17.0,11.313708,12.727922,28.991378,22.627417,0
9,9.0,5.0,11.0,2.0,7.500853,1.0,0.545800,8.5,49.0,29.5,33.0,3.535534,0.000000,27.577164,0.000000,0
